In [3]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import math
import time
import os

In [4]:
class FourierFeatures(nn.Module):
    def __init__(self, in_features, out_features, scale=10.0):
        super(FourierFeatures, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.B = nn.Parameter(scale * torch.randn(in_features, out_features), requires_grad=False)

    def forward(self, x):
        x_proj = 2 * np.pi * x @ self.B
        fourier = torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1)
        return torch.cat([x, fourier], dim=-1) 

In [5]:
class Network(nn.Module):
    def __init__(self, num_input=2, fourier_features=64, layers=[128, 128, 128, 128], num_output=4, scale=10.0):
        super(Network, self).__init__()
        self.fourier = FourierFeatures(num_input, fourier_features, scale)
        input_size = num_input + 2*fourier_features
        self.input_layer = nn.Linear(input_size, layers[0])
        self.hidden_layer = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.hidden_layer.append(nn.Linear(layers[i], layers[i+1]))
        self.output_layer = nn.Linear(layers[-1], num_output)

    def forward(self, x):
        x = self.fourier(x)
        out = torch.tanh(self.input_layer(x))
        for layer in self.hidden_layer:
            out = torch.tanh(layer(out))
        out = self.output_layer(out)
        return out

In [13]:
class Pinns:
    def __init__(self):
        # Transfer to GPU if available
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.network = Network().to(self.device)

        # Physical parameters (dimensionless)
        self.Ra = torch.tensor(1e7, device=self.device)  # Rayleigh number
        self.Pr = torch.tensor(0.7096, device=self.device)  # Prandtl number

        # Domain definition (dimensionless)
        dx = 0.01
        dy = 0.01
        self.x = torch.arange(-0.25, 0.25 + dx, dx)
        self.y = torch.arange(-0.5, 0.5 + dy, dy)
        self.X = torch.stack(torch.meshgrid(self.x, self.y, indexing='ij')).reshape(2, -1).T

        # Transfer tensor to GPU
        self.X = self.X.to(self.device)

        # Error criterion
        self.criterion = nn.MSELoss()

        # Optimizer settings
        self.adam = torch.optim.Adam(self.network.parameters(), lr=1e-5)
        #, lr=1e-4
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.adam, step_size=10000, gamma=0.9)
        self.optimizer = torch.optim.LBFGS(
            self.network.parameters(),
            lr=1.0,
            max_iter=1000,
            max_eval=1000,
            history_size=50,
            tolerance_grad=1e-7,
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"
        )

        # For tracking loss
        self.loss_history = []

    def apply_hard_boundary_conditions(self, predictions):
        u_pred = predictions[:, 0].reshape(len(self.x), len(self.y))
        v_pred = predictions[:, 1].reshape(len(self.x), len(self.y))
        p_pred = predictions[:, 2].reshape(len(self.x), len(self.y))
        theta_pred = predictions[:, 3].reshape(len(self.x), len(self.y))

        # Apply boundary conditions
        # Left wall (x=-0.2): u=0, v=0, theta=0 (cold wall)
        u_pred[0, :] = 0.0
        v_pred[0, :] = 0.0
        theta_pred[0, :] = 0.0

        # Right wall (x=0.2): u=0, v=0, theta=1 (hot wall)
        u_pred[-1, :] = 0.0
        v_pred[-1, :] = 0.0
        theta_pred[-1, :] = 1.0

        # Bottom wall (y=-0.4): u=0, v=0, adiabatic (dtheta/dy=0)
        u_pred[:, 0] = 0.0
        v_pred[:, 0] = 0.0
        theta_pred[:, 0] = theta_pred[:, 1]

        # Top wall (y=0.4): u=0, v=0, adiabatic (dtheta/dy=0)
        u_pred[:, -1] = 0.0
        v_pred[:, -1] = 0.0
        theta_pred[:, -1] = theta_pred[:, -2]

        return u_pred, v_pred, p_pred, theta_pred

    def loss_f(self):
        # Reset gradients
        self.adam.zero_grad()
        self.optimizer.zero_grad()

        # Network prediction
        self.uvpt_pred = self.network(self.X)

        # Apply hard boundary conditions
        self.u_pred, self.v_pred, self.p_pred, self.theta_pred = self.apply_hard_boundary_conditions(self.uvpt_pred)

        dx = self.x[1] - self.x[0]
        dy = self.y[1] - self.y[0]

        # Finite difference operators
        def central_diff_x(arr):
            res = torch.zeros_like(arr)
            res[1:-1, :] = (arr[2:, :] - arr[:-2, :]) / (2 * dx)
            return res

        def central_diff_y(arr):
            res = torch.zeros_like(arr)
            res[:, 1:-1] = (arr[:, 2:] - arr[:, :-2]) / (2 * dy)
            return res

        def second_diff_x(arr):
            res = torch.zeros_like(arr)
            res[1:-1, :] = (arr[2:, :] - 2 * arr[1:-1, :] + arr[:-2, :]) / (dx ** 2)
            return res

        def second_diff_y(arr):
            res = torch.zeros_like(arr)
            res[:, 1:-1] = (arr[:, 2:] - 2 * arr[:, 1:-1] + arr[:, :-2]) / (dy ** 2)
            return res

        # Compute derivatives
        du_dx = central_diff_x(self.u_pred)
        du_dy = central_diff_y(self.u_pred)
        du_dxx = second_diff_x(self.u_pred)
        du_dyy = second_diff_y(self.u_pred)

        dv_dx = central_diff_x(self.v_pred)
        dv_dy = central_diff_y(self.v_pred)
        dv_dxx = second_diff_x(self.v_pred)
        dv_dyy = second_diff_y(self.v_pred)

        dp_dx = central_diff_x(self.p_pred)
        dp_dy = central_diff_y(self.p_pred)

        dtheta_dx = central_diff_x(self.theta_pred)
        dtheta_dy = central_diff_y(self.theta_pred)
        dtheta_dxx = second_diff_x(self.theta_pred)
        dtheta_dyy = second_diff_y(self.theta_pred)

        # Extract interior points (exclude boundaries)
        slice_in = slice(1, -1)
        u_int = self.u_pred[slice_in, slice_in]
        v_int = self.v_pred[slice_in, slice_in]
        theta_int = self.theta_pred[slice_in, slice_in]

        du_dx_int = du_dx[slice_in, slice_in]
        du_dy_int = du_dy[slice_in, slice_in]
        du_dxx_int = du_dxx[slice_in, slice_in]
        du_dyy_int = du_dyy[slice_in, slice_in]

        dv_dx_int = dv_dx[slice_in, slice_in]
        dv_dy_int = dv_dy[slice_in, slice_in]
        dv_dxx_int = dv_dxx[slice_in, slice_in]
        dv_dyy_int = dv_dyy[slice_in, slice_in]

        dp_dx_int = dp_dx[slice_in, slice_in]
        dp_dy_int = dp_dy[slice_in, slice_in]

        dtheta_dx_int = dtheta_dx[slice_in, slice_in]
        dtheta_dy_int = dtheta_dy[slice_in, slice_in]
        dtheta_dxx_int = dtheta_dxx[slice_in, slice_in]
        dtheta_dyy_int = dtheta_dyy[slice_in, slice_in]

        # Dimensionless governing equations for natural convection
        # Continuity equation:
        continuity_eq = du_dx_int + dv_dy_int

        # X-momentum equation:
        u_momentum_eq = (u_int * du_dx_int + v_int * du_dy_int + dp_dx_int -
                        torch.sqrt(self.Pr / self.Ra) * (du_dxx_int + du_dyy_int))

        # Y-momentum equation:
        v_momentum_eq = (u_int * dv_dx_int + v_int * dv_dy_int + dp_dy_int -
                        torch.sqrt(self.Pr / self.Ra) * (dv_dxx_int + dv_dyy_int) - theta_int)

        # Energy equation:
        energy_eq = (u_int * dtheta_dx_int + v_int * dtheta_dy_int -
                    torch.sqrt(1.0 / (self.Ra * self.Pr)) * (dtheta_dxx_int + dtheta_dyy_int))

        # Compute total loss
        self.loss = (self.criterion(continuity_eq.reshape(-1), torch.zeros_like(continuity_eq).reshape(-1)) +
                    self.criterion(u_momentum_eq.reshape(-1), torch.zeros_like(u_momentum_eq).reshape(-1)) +
                    self.criterion(v_momentum_eq.reshape(-1), torch.zeros_like(v_momentum_eq).reshape(-1)) +
                    self.criterion(energy_eq.reshape(-1), torch.zeros_like(energy_eq).reshape(-1)))

        self.loss.backward()
        return self.loss

    def train(self, num_epochs=1):
        self.network.train()
        for i in range(num_epochs):
            loss = self.loss_f()
            self.loss_history.append(loss.item())
            if i % 10 == 0:
                print(f"Iteration {i}, Loss: {self.loss.item():.9f}")
            if i % 100 == 0:
                torch.save(self.network.state_dict(), r'C:/Users/hossein/Result/Natural Convection/Ra=10^7/2-FF-FD-PINNs/model')
            self.adam.step(self.loss_f)
        self.optimizer.step(self.loss_f)
        torch.save(self.network.state_dict(), r'C:/Users/hossein/Result/Natural Convection/Ra=10^7/2-FF-FD-PINNs/model')

    def plot(self):
        self.network.eval()
        with torch.no_grad():
            self.u = self.u_pred.cpu().numpy().T
            self.v = self.v_pred.cpu().numpy().T
            self.p = self.p_pred.cpu().numpy().T
            self.theta = self.theta_pred.cpu().numpy().T

        save_dir = "C:/Users/hossein/Result/Natural Convection/Ra=10^7/2-FF-FD-PINNs"
        plt.rcParams.update({
            "font.family": "Times New Roman",
            "font.style": "italic"
        })

        # Temperature field
        plt.figure(figsize=(4, 5))
        contour1 = plt.contourf(self.x.detach().cpu().numpy(), self.y.detach().cpu().numpy(),
                               self.theta, levels=50, cmap="jet")
        plt.colorbar(contour1, aspect=60)
        plt.xlabel("x", fontsize=14)
        plt.ylabel("y", fontsize=14)
        plt.title("Temperature Field (θ)", fontsize=14)
        plt.savefig(os.path.join(save_dir, "Temperature.png"), dpi=300)
        plt.show()
        plt.close()

        # U velocity
        plt.figure(figsize=(4, 5))
        contour2 = plt.contourf(self.x.detach().cpu().numpy(), self.y.detach().cpu().numpy(),
                               self.u, levels=50, cmap="jet")
        plt.colorbar(contour2, aspect=60)
        plt.xlabel("x", fontsize=14)
        plt.ylabel("y", fontsize=14)
        plt.title("u-velocity", fontsize=14)
        plt.savefig(os.path.join(save_dir, "u-velocity.png"), dpi=300)
        plt.show()
        plt.close()
        
        # V velocity
        plt.figure(figsize=(4, 5))
        contour3 = plt.contourf(self.x.detach().cpu().numpy(), self.y.detach().cpu().numpy(),
                               self.v, levels=50, cmap="jet")
        plt.colorbar(contour3, aspect=60)
        plt.xlabel("x", fontsize=14)
        plt.ylabel("y", fontsize=14)
        plt.title("v-velocity", fontsize=14)
        plt.savefig(os.path.join(save_dir, "v-velocity.png"), dpi=300)
        plt.show()
        plt.close()

        # Streamlines
        fig, ax = plt.subplots(figsize=(4, 6))
        ax.streamplot(self.x.detach().cpu().numpy(), self.y.detach().cpu().numpy(),
                      self.u, self.v, color='red', cmap='autumn', linewidth=0.5, density=2, arrowsize=0.6)
        ax.set_xlabel("x", fontsize=14)
        ax.set_ylabel("y", fontsize=14)
        ax.set_title("Streamlines", fontsize=14)
        ax.set_xlim(-0.2, 0.2)
        ax.set_ylim(-0.4, 0.4)
        fig.savefig(os.path.join(save_dir, "Streamlines.png"), dpi=300)
        plt.show()

    def plot_loss(self):
        plt.figure(figsize=(10, 6))
        plt.semilogy(self.loss_history)
        plt.xlabel('Iteration')
        plt.ylabel('Loss (Log Scale)')
        plt.title('Training Loss History')
        plt.grid(True)
        plt.show()
        df = pd.DataFrame({"loss": self.loss_history})
        df.to_csv("C:/Users/hossein/Result/Natural Convection/Ra=10^7/2-FF-FD-PINNs/loss_history5.csv", index=False)

In [12]:
net = Pinns()
net.network.load_state_dict(torch.load(r'C:/Users/hossein/Result/Natural Convection/Ra=10^7/2-FF-FD-PINNs/model'))
net.train(num_epochs=100000)
net.plot()
net.plot_loss()

Iteration 0, Loss: 0.000000008
Iteration 10, Loss: 0.000000293
Iteration 20, Loss: 0.000000098
Iteration 30, Loss: 0.000000041
Iteration 40, Loss: 0.000000020
Iteration 50, Loss: 0.000000012
Iteration 60, Loss: 0.000000009
Iteration 70, Loss: 0.000000008
Iteration 80, Loss: 0.000000008
Iteration 90, Loss: 0.000000008
Iteration 100, Loss: 0.000000008
Iteration 110, Loss: 0.000000008
Iteration 120, Loss: 0.000000008
Iteration 130, Loss: 0.000000008
Iteration 140, Loss: 0.000000008
Iteration 150, Loss: 0.000000008
Iteration 160, Loss: 0.000000008
Iteration 170, Loss: 0.000000008
Iteration 180, Loss: 0.000000008
Iteration 190, Loss: 0.000000008
Iteration 200, Loss: 0.000000008
Iteration 210, Loss: 0.000000008
Iteration 220, Loss: 0.000000008
Iteration 230, Loss: 0.000000008
Iteration 240, Loss: 0.000000008
Iteration 250, Loss: 0.000000008


KeyboardInterrupt: 